In [1]:
# TO BE REMOVED!!!
%load_ext autoreload
%autoreload 2
import os, shutil
import nglview as nv
import ipywidgets
import zipfile
import webbrowser

antibody_id = '4G6K'
antigen_id  = '4I1B'
complex_id  = '4C6M'
out_path = 'data/antibody/'

In [2]:
# Helpers
def def_dict(propierties={}):
    def_props = {'out_log_path': 'log/log.log',
                 'err_log_path': 'log/log.err',
                 'remove_tmp': True,
                 'can_write_console_log': True}
    def_props.update(propierties)
    return def_props

def show_pdbs(pdbs, surface=False):
    # Load the PDB files
    views = [nv.show_file(pdb) for pdb in pdbs]
    for view in views:
        if surface:
            view.clear()
            view.add_cartoon(color='black')
            view.add_surface(color='electrostatic', opacity=0.5)
        view.layout.width = '100%'
    return ipywidgets.HBox(views)

def display_actpass(pdb, actpass, opacity=1):
    with open(actpass, 'r') as file:
        actpass = file.read().splitlines()
        act_res = actpass[0].replace(' ', ', ')
        pas_res = actpass[1].replace(' ', ', ')
        
    # Load the PDB files
    view = nv.NGLWidget()
    view.add_component(pdb)
    view.clear()
    view.add_cartoon(color='black')
    view.add_ball_and_stick(color='grey',opacity=opacity)
    view.add_surface(selection=f'not ( {pas_res}, {act_res} )', color='white', opacity=opacity)
    if act_res != '':
        view.add_surface(selection=f'{act_res}', color='red')
    if pas_res != '':
        view.add_surface(selection=f'{pas_res}', color='green', opacity=opacity)
    view.layout.width = '100%'
    return view

# Antibody use case

***
This tutorial aims to illustrate the process of 
***


## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_model](https://github.com/bioexcel/biobb_model): Tools to model macromolecular structures.
 - [biobb_gromacs](https://github.com/bioexcel/biobb_gromacs): Tools to setup and run Molecular Dynamics simulations.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
 - [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_md_setup.git
cd biobb_wf_md_setup
conda env create -f conda_env/environment.yml
conda activate biobb_GMX_MDsetup_tutorial
jupyter nbextension enable python-markdown/main
jupyter-notebook biobb_wf_md_setup/notebooks/biobb_MDsetup_tutorial.ipynb
```

Please execute the following commands before launching the Jupyter Notebook if you experience some issues with widgets such as NGL View (3D molecular visualization):

```console
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
```

***


## Pipeline steps


 1. [Input Parameters](#input)
 2. [Fetching PDB Structure](#fetch)
 3. [Fix Protein Structure](#fix)
 4. [Create Protein System Topology](#top)
 5. [Create Solvent Box](#box)
 6. [Fill the Box with Water Molecules](#water)
 7. [Adding Ions](#ions)
 8. [Energetically Minimize the System](#min)
 9. [Equilibrate the System (NVT)](#nvt)
 10. [Equilibrate the System (NPT)](#npt)
 11. [Free Molecular Dynamics Simulation](#free)
 12. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 13. [Output Files](#output)
 14. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
    title="Bioexcel2 logo" width="400" />
***


## Sampling using ensemble from MD with GROMACS

In [ ]:
# https://mmb.irbbarcelona.org/biobb/workflows/tutorials/biobb_wf_flexdyn ?
# https://github.com/bioexcel/biobb_wf_flexdyn
# https://github.com/alevil-gmx/workflow_template/blob/main/workflow_template/workshop-template-ab.ipynb

<a id="input"></a>
### Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 1AKI)

In [3]:
cwd = os.getcwd()
MD_dir = out_path+'1_MD/'
MD_inp = MD_dir+'input/'
os.makedirs(MD_inp, exist_ok=True) 
pdbCode = "4G6K"

<a id="fetch"></a>
***
### Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the RCSB PDB database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

In [5]:
# Downloading desired PDB file 
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = f'{MD_dir}{pdbCode}.pdb'
prop = {
    'pdb_code': pdbCode
}

# Create and launch bb
pdb(output_pdb_path=downloaded_pdb,
    properties=def_dict(prop))

2025-05-19 16:20:45,534 [MainThread  ] [INFO ]  Module: biobb_io.api.pdb Version: 5.0.1
2025-05-19 16:20:45,534 [MainThread  ] [INFO ]  Downloading 4g6k from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb4g6k.ent
2025-05-19 16:20:45,938 [MainThread  ] [INFO ]  Writting pdb to: data/antibody/1_MD/4G6K.pdb
2025-05-19 16:20:45,938 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']
2025-05-19 16:20:45,944 [MainThread  ] [INFO ]  


0

<a id="vis3D"></a>
#### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**:    

In [13]:
# Show protein
view = nv.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="fix"></a>
***
### Fix protein structure
**Checking** and **fixing** (if needed) the protein structure:<br>
- **Modeling** **missing side-chain atoms**, modifying incorrect **amide assignments**, choosing **alternative locations**.<br>
- **Checking** for missing **backbone atoms**, **heteroatoms**, **modified residues** and possible **atomic clashes**.

***
**Building Blocks** used:
 - [FixSideChain](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_side_chain) from **biobb_model.model.fix_side_chain**
***

In [6]:
# Check & Fix PDB
# Import module
from biobb_model.model.fix_side_chain import fix_side_chain

# Create prop dict and inputs/outputs
fixed_pdb = f'{MD_dir}{pdbCode}_fixed.pdb'

# Create and launch bb
fix_side_chain(input_pdb_path=downloaded_pdb, 
               output_pdb_path=fixed_pdb,
               properties=def_dict())

2025-05-19 16:20:50,662 [MainThread  ] [INFO ]  Module: biobb_model.model.fix_side_chain Version: 5.0.0
2025-05-19 16:20:50,663 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_6721d9c4-8255-4f05-95c6-509413aa75ba directory successfully created
2025-05-19 16:20:50,664 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_6721d9c4-8255-4f05-95c6-509413aa75ba
2025-05-19 16:20:50,665 [MainThread  ] [INFO ]  check_structure -i /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_6721d9c4-8255-4f05-95c6-509413aa75ba/4G6K.pdb -o /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_6721d9c4-8255-4f05-95c6-509413aa75ba/4G6K_fixed.pdb --force_save fixside --fix ALL

2025-05-19 16:20:50,914 [MainThread  ] [INFO ]  Executing: check_structure -i /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_h

0

#### Visualizing 3D structure
Visualizing the fixed **PDB structure** using **NGL**. In this particular example, the checking step didn't find any issue to be solved, so there is no difference between the original structure and the fixed one.   

In [7]:
# Show protein
view = nv.show_structure_file(fixed_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="top"></a>
***
### Create protein system topology
**Building GROMACS topology** corresponding to the protein structure.<br>
Force field used in this tutorial is [**amber99sb-ildn**](https://dx.doi.org/10.1002%2Fprot.22711): AMBER **parm99** force field with **corrections on backbone** (sb) and **side-chain torsion potentials** (ildn). Water molecules type used in this tutorial is [**spc/e**](https://pubs.acs.org/doi/abs/10.1021/j100308a038).<br>
Adding **hydrogen atoms** if missing. Automatically identifying **disulfide bridges**. <br>

Generating two output files: 
- **GROMACS structure** (gro file)
- **GROMACS topology** ZIP compressed file containing:
    - *GROMACS topology top file* (top file)
    - *GROMACS position restraint file/s* (itp file/s)
***
**Building Blocks** used:
 - [Pdb2gmx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.pdb2gmx) from **biobb_gromacs.gromacs.pdb2gmx**
***

In [8]:
# Downloading CHARMM36 FF
!curl  -o {MD_inp}charmm36-jul2017.ff.tgz https://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/charmm36-jul2017.ff.tgz
# Unzip CHARMM36 FF
!tar -xvzf {MD_inp}charmm36-jul2017.ff.tgz -C {MD_inp}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  586k  100  586k    0     0   119k      0  0:00:04  0:00:04 --:--:--  125k
charmm36-jul2017.ff/
charmm36-jul2017.ff/spc.itp
charmm36-jul2017.ff/ffbonded.itp
charmm36-jul2017.ff/merged.rtp
charmm36-jul2017.ff/nbfix.itp
charmm36-jul2017.ff/gb.itp
charmm36-jul2017.ff/forcefield.doc
charmm36-jul2017.ff/merged.hdb
charmm36-jul2017.ff/ions.itp
charmm36-jul2017.ff/merged.arn
charmm36-jul2017.ff/merged.vsd
charmm36-jul2017.ff/spce.itp
charmm36-jul2017.ff/old_c36_cmap.itp
charmm36-jul2017.ff/ffnonbonded.itp
charmm36-jul2017.ff/atomtypes.atp
charmm36-jul2017.ff/merged.c.tdb
charmm36-jul2017.ff/forcefield.itp
charmm36-jul2017.ff/watermodels.dat
charmm36-jul2017.ff/tip4p.itp
charmm36-jul2017.ff/cmap.itp
charmm36-jul2017.ff/merged.n.tdb
charmm36-jul2017.ff/tip3p.itp
charmm36-jul2017.ff/merged.r2b


In [12]:
# Create system topology
# Import module
from biobb_gromacs.gromacs.pdb2gmx import pdb2gmx

# Create inputs/outputs
output_pdb2gmx_gro     = f'{MD_dir}{pdbCode}__pdb2gmx.gro'
output_pdb2gmx_top_zip = f'{MD_dir}{pdbCode}__pdb2gmx_top.zip'

prop = {
    'force_field' : 'charmm36-jul2017',
    'gmx_lib' : f'{cwd}/{MD_inp}',
    'water_type' : 'tip3p',
    #'force_field' : 'amber99sb-ildn'
}

# Create and launch bb
pdb2gmx(input_pdb_path=fixed_pdb, 
        output_gro_path=output_pdb2gmx_gro, 
        output_top_zip_path=output_pdb2gmx_top_zip,
        properties=def_dict(prop))

2025-05-19 16:22:50,532 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.pdb2gmx Version: 5.0.0
2025-05-19 16:22:50,532 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e7614182-e452-435c-bead-25fcc027d9c1 directory successfully created
2025-05-19 16:22:50,533 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_fixed.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e7614182-e452-435c-bead-25fcc027d9c1
2025-05-19 16:22:50,533 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e7614182-e452-435c-bead-25fcc027d9c1/4G6K_fixed.pdb -o /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e7614182-e452-435c-bead-25fcc027d9c1/4G6K__pdb2gmx.gro -p p2g.top -water tip3p -ff charmm36-jul2017 -i posre.itp

2025-05-19 16:22:51,144 [MainThread  ] [INFO ]  Executing: gmx -nobackup -no

0

#### Visualizing 3D structure
Visualizing the generated **GRO structure** using **NGL**. Note that **hydrogen atoms** were added to the structure by the **pdb2gmx GROMACS tool** when generating the **topology**.    

In [42]:
# Show protein
view = nv.show_structure_file(output_pdb2gmx_gro)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="box"></a>
***
### Create solvent box
Define the unit cell for the **protein structure MD system** to fill it with water molecules.<br>
A **cubic box** is used to define the unit cell, with a **distance from the protein to the box edge of 1.0 nm**. The protein is **centered in the box**.  

***
**Building Blocks** used:
 - [Editconf](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.editconf) from **biobb_gromacs.gromacs.editconf** 
***

In [13]:
# Editconf: Create solvent box
# Import module
from biobb_gromacs.gromacs.editconf import editconf

# Create prop dict and inputs/outputs
output_editconf_gro = f'{MD_dir}{pdbCode}_editconf.gro'

prop = {
    'box_type': 'dodecahedron',
    'distance_to_molecule': 0.7
}

#Create and launch bb
editconf(input_gro_path=output_pdb2gmx_gro, 
         output_gro_path=output_editconf_gro,
         properties=def_dict(prop))

2025-05-19 16:23:00,202 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.editconf Version: 5.0.0
2025-05-19 16:23:00,202 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_b42d8fa3-1809-46bc-ada3-9b815b053336 directory successfully created
2025-05-19 16:23:00,203 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K__pdb2gmx.gro to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_b42d8fa3-1809-46bc-ada3-9b815b053336
2025-05-19 16:23:00,203 [MainThread  ] [INFO ]  Distance of the box to molecule:   0.70
2025-05-19 16:23:00,203 [MainThread  ] [INFO ]  Centering molecule in the box.
2025-05-19 16:23:00,204 [MainThread  ] [INFO ]  Box type: dodecahedron
2025-05-19 16:23:00,204 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright editconf -f /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_b42d8fa3-1809-46bc-ada3-9b815b053336/4G6K__pdb2gmx.gro -o /home/rchaves/repo/ab_des

0

<a id="water"></a>
***
### Fill the box with water molecules
Fill the unit cell for the **protein structure system** with water molecules.<br>
The solvent type used is the default **Simple Point Charge water (SPC)**, a generic equilibrated 3-point solvent model. 

***
**Building Blocks** used:
 - [Solvate](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.solvate) from **biobb_gromacs.gromacs.solvate** 
***

In [14]:
# Solvate: Fill the box with water molecules
from biobb_gromacs.gromacs.solvate import solvate

# Create prop dict and inputs/outputs
output_solvate_gro     = f'{MD_dir}{pdbCode}_solvate.gro'
output_solvate_top_zip = f'{MD_dir}{pdbCode}_solvate_top.zip'

# Create and launch bb
solvate(input_solute_gro_path=output_editconf_gro, 
        output_gro_path=output_solvate_gro, 
        input_top_zip_path=output_pdb2gmx_top_zip, 
        output_top_zip_path=output_solvate_top_zip,
        properties=def_dict(prop))

2025-05-19 16:23:07,444 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.solvate Version: 5.0.0
2025-05-19 16:23:07,445 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0ffea171-ee61-47cf-a415-5195f61b94ba directory successfully created
2025-05-19 16:23:07,446 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_editconf.gro to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0ffea171-ee61-47cf-a415-5195f61b94ba
2025-05-19 16:23:07,449 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K__pdb2gmx_top.zip
2025-05-19 16:23:07,449 [MainThread  ] [INFO ]  to:
2025-05-19 16:23:07,449 [MainThread  ] [INFO ]  ['2b88df63-9a0a-4df6-b090-b488608448fc/p2g.top', '2b88df63-9a0a-4df6-b090-b488608448fc/p2g_Protein_chain_H.itp', '2b88df63-9a0a-4df6-b090-b488608448fc/p2g_Protein_chain_L.itp', '2b88df63-9a0a-4df6-b090-b488608448fc/p

/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/biobb_common/generic/biobb_object.py:183: UserWarning: Warning: distance_to_molecule is not a recognized property. The most similar property is: stage_io_dict
  warnings.warn(
/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/biobb_common/generic/biobb_object.py:183: UserWarning: Warning: box_type is not a recognized property. The most similar property is: sandbox_path
  warnings.warn(


2025-05-19 16:23:07,670 [MainThread  ] [INFO ]  Executing: gmx -nobackup -nocopyright solvate -cp /home/rchaves/repo/ab_design/biobb_wf_had...
2025-05-19 16:23:07,670 [MainThread  ] [INFO ]  Exit code: 0
2025-05-19 16:23:07,671 [MainThread  ] [INFO ]  
         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
------

0

#### Visualizing 3D structure
Visualizing the **protein system** with the newly added **solvent box** using **NGL**.<br> Note the **cubic box** filled with **water molecules** surrounding the **protein structure**, which is **centered** right in the middle of the cube.

In [47]:
# Show protein
view = nv.show_structure_file(output_solvate_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='green')
view.add_representation(repr_type='ball+stick', selection='SOL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="ions"></a>
***
### Adding ions
Add ions to neutralize the **protein structure** charge
- [Step 1](#ionsStep1): Creating portable binary run file for ion generation
- [Step 2](#ionsStep2): Adding ions to **neutralize** the system
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Genion](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.genion) from **biobb_gromacs.gromacs.genion** 
***

<a id="ionsStep1"></a>
#### Step 1: Creating portable binary run file for ion generation
A simple **energy minimization** molecular dynamics parameters (mdp) properties will be used to generate the portable binary run file for **ion generation**, although **any legitimate combination of parameters** could be used in this step.

In [15]:
# Grompp: Creating portable binary run file for ion generation
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppion_tpr = f'{MD_dir}{pdbCode}_gppion.tpr'
prop = {
    'simulation_type': 'ions',
    'gmx_lib' : f'{cwd}/{MD_inp}',
    'maxwarn': 1
}

# Create and launch bb
grompp(input_gro_path=output_solvate_gro, 
       input_top_zip_path=output_solvate_top_zip, 
       output_tpr_path=output_gppion_tpr,  
       properties=def_dict(prop))

2025-05-19 16:23:27,987 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.grompp Version: 5.0.0
2025-05-19 16:23:27,988 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0e856f46-8902-4bce-a948-17e63baadb3e directory successfully created
2025-05-19 16:23:27,990 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_solvate.gro to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0e856f46-8902-4bce-a948-17e63baadb3e
2025-05-19 16:23:27,994 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_solvate_top.zip
2025-05-19 16:23:27,994 [MainThread  ] [INFO ]  to:
2025-05-19 16:23:27,995 [MainThread  ] [INFO ]  ['/home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0e856f46-8902-4bce-a948-17e63baadb3e/p2g.top', '/home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0e856f4

0

<a id="ionsStep2"></a>
#### Step 2: Adding ions to neutralize the system
Replace **solvent molecules** with **ions** to **neutralize** the system.

In [16]:
# Genion: Adding ions to neutralize the system
from biobb_gromacs.gromacs.genion import genion

# Create prop dict and inputs/outputs
output_genion_gro     = f'{MD_dir}{pdbCode}_genion.gro'
output_genion_top_zip = f'{MD_dir}{pdbCode}_genion_top.zip'
prop={
    'neutral':True,
    'ionic_concentration' : 0.15
}

# Create and launch bb
genion(input_tpr_path=output_gppion_tpr, 
       output_gro_path=output_genion_gro, 
       input_top_zip_path=output_solvate_top_zip, 
       output_top_zip_path=output_genion_top_zip, 
       properties=def_dict(prop))

2025-05-19 16:23:34,283 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.genion Version: 5.0.0
2025-05-19 16:23:34,284 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_eb8effc8-5f9f-4f15-a082-9663b49ff874 directory successfully created
2025-05-19 16:23:34,287 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_gppion.tpr to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_eb8effc8-5f9f-4f15-a082-9663b49ff874
2025-05-19 16:23:34,287 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/8e08cfb8-9abf-4ea6-bb24-315ffd030804.stdin to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_eb8effc8-5f9f-4f15-a082-9663b49ff874
2025-05-19 16:23:34,290 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_solvate_top.zip
2025-05-19 16:23:34,290 [Main

/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/biobb_common/generic/biobb_object.py:183: UserWarning: Warning: ionic_concentration is not a recognized property. The most similar property is: concentration
  warnings.warn(


0

#### Visualizing 3D structure
Visualizing the **neutralized protein system** with the newly added **ions** using **NGL**

In [51]:
# Show protein
view = nv.show_structure_file(output_genion_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='sstruc')
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="min"></a>
***
### Energetically minimize the system
Energetically minimize the **protein system** till reaching a desired potential energy.
- [Step 1](#emStep1): Creating portable binary run file for energy minimization
- [Step 2](#emStep2): Energetically minimize the **system** till reaching a force of 500 kJ mol-1 nm-1.
- [Step 3](#emStep3): Checking **energy minimization** results. Plotting energy by time during the **minimization** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="emStep1"></a>
#### Step 1: Creating portable binary run file for energy minimization
The **minimization** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **energy minimization**:

-  integrator  = steep ; Algorithm (steep = steepest descent minimization)
-  emtol       = 1000.0 ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01 ; Minimization step size (nm)
-  nsteps      = 50000 ; Maximum number of (minimization) steps to perform

In this particular example, the method used to run the **energy minimization** is the default **steepest descent**, but the **maximum force** is placed at **500 KJ/mol\*nm^2**, and the **maximum number of steps** to perform (if the maximum force is not reached) to **5,000 steps**. 

In [19]:
# https://github.com/alevil-gmx/workflow_template/blob/main/workflow_template/data/input/emin-charmm.mdp
input_mdp_min = f"{MD_inp}emin-charmm.mdp"
fl = """title       = CHARMM steepest descent enrgy minimisation

; Parameters describing what to do, when to stop and what to save
integrator  = steep  ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0 ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01   ; Minimization step size
nstenergy   = 500    ; save energies every 1.0 ps, so we can observe if we are successful
nsteps      = -1     ; run as long as we need
; Settings that make sure we run with parameters in harmony with the selected force-field
constraints             = h-bonds   ; bonds involving H are constrained
rcoulomb                = 1.2       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.2       ; short-range van der Waals cutoff (in nm)
vdw-modifier            = Force-switch ;  specific CHARMM
rvdw_switch             = 1.0       ;
DispCorr                = EnerPres  ; account for cut-off vdW scheme
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
fourierspacing          = 0.15     ; grid spacing for FFT
"""

with open(input_mdp_min, 'w') as f:
    f.write(fl)

In [21]:
# Grompp: Creating portable binary run file for mdrun
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppmin_tpr = f'{MD_dir}{pdbCode}_gppmin.tpr'

prop = {
    'gmx_lib' : f'{cwd}/{MD_inp}',
}

# Create and launch bb
grompp(input_gro_path=output_genion_gro, 
       input_top_zip_path=output_genion_top_zip, 
       input_mdp_path=input_mdp_min,
       output_tpr_path=output_gppmin_tpr,  
       properties=def_dict(prop))

2025-05-19 16:25:07,155 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.grompp Version: 5.0.0
2025-05-19 16:25:07,156 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_5a2f2751-4a29-4eb1-b837-f08578d83dfc directory successfully created
2025-05-19 16:25:07,158 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_genion.gro to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_5a2f2751-4a29-4eb1-b837-f08578d83dfc
2025-05-19 16:25:07,159 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/input/emin-charmm.mdp to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_5a2f2751-4a29-4eb1-b837-f08578d83dfc
2025-05-19 16:25:07,163 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_genion_top.zip
2025-05-19 16:25:07,163 [MainThread  ] [INFO ]  to:
2025-05-19 16:25:07,163 [MainThread  ] [INFO ]  ['/ho

0

<a id="emStep2"></a>
#### Step 2: Running Energy Minimization
Running **energy minimization** using the **tpr file** generated in the previous step. 

In [22]:
# Mdrun: Running minimization
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_min_trr = f'{MD_dir}{pdbCode}_min.trr'
output_min_gro = f'{MD_dir}{pdbCode}_min.gro'
output_min_edr = f'{MD_dir}{pdbCode}_min.edr'
output_min_log = f'{MD_dir}{pdbCode}_min.log'

# Create and launch bb
mdrun(input_tpr_path=output_gppmin_tpr, 
      output_trr_path=output_min_trr, 
      output_gro_path=output_min_gro, 
      output_edr_path=output_min_edr, 
      output_log_path=output_min_log,
      properties=def_dict(prop))

2025-05-19 16:25:18,735 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.mdrun Version: 5.0.0
2025-05-19 16:25:18,736 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_cf1fc660-e862-481c-8710-56666593389a directory successfully created
2025-05-19 16:25:18,738 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_gppmin.tpr to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_cf1fc660-e862-481c-8710-56666593389a
2025-05-19 16:25:18,738 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -o /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_cf1fc660-e862-481c-8710-56666593389a/4G6K_min.trr -s /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_cf1fc660-e862-481c-8710-56666593389a/4G6K_gppmin.tpr -c /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_cf1fc660-e862-481c-8710-56666593389a/4G6K_min.gro -e /home

0

<a id="emStep3"></a>
#### Step 3: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the minimization process. 

In [23]:
# GMXEnergy: Getting system energy by time  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_min_ene_xvg = f'{MD_dir}{pdbCode}_min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_min_edr, 
          output_xvg_path=output_min_ene_xvg, 
          properties=def_dict(prop))

2025-05-19 16:25:38,853 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_energy Version: 5.0.1
2025-05-19 16:25:38,853 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_30102591-632e-46c6-a3fd-4fc02fb479e7 directory successfully created
2025-05-19 16:25:38,854 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_min.edr to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_30102591-632e-46c6-a3fd-4fc02fb479e7
2025-05-19 16:25:38,855 [MainThread  ] [INFO ]  gmx energy -f /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_30102591-632e-46c6-a3fd-4fc02fb479e7/4G6K_min.edr -o /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_30102591-632e-46c6-a3fd-4fc02fb479e7/4G6K_min_ene.xvg -xvg none < 66740b61-b7f9-458b-8eb2-d68f6523457f/instructions.in

2025-05-19 16:25:38,8

0

In [24]:
import plotly
import plotly.graph_objs as go

#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_min_ene_xvg,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy KJ/mol-1")
                       )
}

plotly.offline.iplot(fig)

<a id="npt"></a>
***
### Equilibrate the system (NPT)
Equilibrate the **protein system** in **NPT** ensemble (constant Number of particles, Pressure and Temperature).
- [Step 1](#eqNPTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNPTStep2): Equilibrate the **protein system** with **NPT** ensemble.
- [Step 3](#eqNPTStep3): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNPTStep1"></a>
#### Step 1: Creating portable binary run file for system equilibration (NPT)

The **npt** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **NPT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl = Parrinello-Rahman
-  pcoupltype = isotropic
-  tau_p = 1.0
-  ref_p = 1.0
-  compressibility = 4.5e-5
-  refcoord_scaling = com
-  gen_vel = no

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, **5,000 equilibration steps** with the protein **heavy atoms restrained**, and a Parrinello-Rahman **pressure coupling** algorithm.

*Please note that for the sake of time this tutorial is only running 10ps of NPT equilibration, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/07_equil2.html) the simulated time was 100ps.*

In [25]:
# From https://github.com/alevil-gmx/workflow_template/blob/main/workflow_template/data/input/md_eq_posre_charmm36m.mdp
# !! se cambia define -> Define y ns_type -> ns-type o el biobb da error (TO-DO arreglar)

input_mdp_eq = f"{MD_inp}md_eq_posre_charmm36m.mdp"
fl="""Define                   = -DPOSRES
integrator               = md
tinit                    = 0
dt                       = 0.002
nsteps                   = 500000   ; 1  ns
comm_grps                = system
nstxout                  = 0
nstvout                  = 0
nstfout                  = 0
nstlog                   = 10000
nstenergy                = 10000
nstxout-compressed       = 10000
compressed-x-grps        = system 
nstlist                  = 10
ns-type                  = grid
pbc                      = xyz
; 
coulombtype              = PME
rcoulomb                 = 1.2
fourierspacing           = 0.15 
;
vdwtype                  = Cut-off
vdw_modifier=Force-switch
rvdw_switch              = 1.0
rvdw                     = 1.2
DispCorr                 = EnerPres
;
constraints              = h-bonds
constraint_algorithm     = LINCS
; new
Pcoupl                   = Berendsen
tau_p                    = 5.0  
ref_p                    = 1.0 
compressibility          = 4.5e-5 
refcoord-scaling         = all     ; to use with pos restrain
Tcoupl                   = v-rescale 
tc-grps                  = system
tau_t                    = 0.5 
ref_t                    = 298 """
with open(input_mdp_eq, 'w') as f:
    f.write(fl)

In [26]:
# Grompp: Creating portable binary run file for NPT Equilibration
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnpt_tpr = f'{MD_dir}{pdbCode}_gppnpt.tpr'

prop = {
    'mdp': {
        'nsteps' : 5000, # subir
    },
    'gmx_lib' : f'{cwd}/{MD_inp}',
    'maxwarn' : 1,
}

# Create and launch bb
grompp(input_gro_path=output_min_gro, 
       input_top_zip_path=output_genion_top_zip, 
       input_mdp_path=input_mdp_eq,
       output_tpr_path=output_gppnpt_tpr,  
       properties=def_dict(prop))

2025-05-19 16:25:48,448 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.grompp Version: 5.0.0
2025-05-19 16:25:48,449 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_6fffce82-0cf1-4705-8321-8ddb2478ee8a directory successfully created
2025-05-19 16:25:48,451 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_min.gro to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_6fffce82-0cf1-4705-8321-8ddb2478ee8a
2025-05-19 16:25:48,452 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/input/md_eq_posre_charmm36m.mdp to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_6fffce82-0cf1-4705-8321-8ddb2478ee8a
2025-05-19 16:25:48,455 [MainThread  ] [INFO ]  Extracting: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_genion_top.zip
2025-05-19 16:25:48,455 [MainThread  ] [INFO ]  to:
2025-05-19 16:25:48,456 [MainThread  ] [INFO ]

0

<a id="eqNPTStep2"></a>
#### Step 2: Running NPT equilibration

In [27]:
# Mdrun: Running NPT System Equilibration
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_npt_trr = f'{MD_dir}{pdbCode}_npt.trr'
output_npt_gro = f'{MD_dir}{pdbCode}_npt.gro'
output_npt_edr = f'{MD_dir}{pdbCode}_npt.edr'
output_npt_log = f'{MD_dir}{pdbCode}_npt.log'
output_npt_cpt = f'{MD_dir}{pdbCode}_npt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnpt_tpr, 
      output_trr_path=output_npt_trr, 
      output_gro_path=output_npt_gro, 
      output_edr_path=output_npt_edr, 
      output_log_path=output_npt_log, 
      output_cpt_path=output_npt_cpt,
      properties=def_dict())

2025-05-19 16:26:10,219 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.mdrun Version: 5.0.0
2025-05-19 16:26:10,219 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_95938b75-a977-49de-8c7d-abd14ba51677 directory successfully created
2025-05-19 16:26:10,222 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_gppnpt.tpr to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_95938b75-a977-49de-8c7d-abd14ba51677
2025-05-19 16:26:10,223 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -o /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_95938b75-a977-49de-8c7d-abd14ba51677/4G6K_npt.trr -s /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_95938b75-a977-49de-8c7d-abd14ba51677/4G6K_gppnpt.tpr -c /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_95938b75-a977-49de-8c7d-abd14ba51677/4G6K_npt.gro -e /home

/home/rchaves/miniforge3/envs/biobb_wf_antibody/lib/python3.10/site-packages/biobb_common/tools/file_utils.py:770: UserWarning:

Path data/antibody/1_MD/4G6K_npt.trr --- biobb_gromacs.gromacs.mdrun: Unexisting output_trr_path file.



0

<a id="eqNPTStep3"></a>
#### Step 3: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [28]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_npt_pd_xvg = f'{MD_dir}{pdbCode}_npt_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_npt_edr, 
          output_xvg_path=output_npt_pd_xvg, 
          properties=prop)

2025-05-19 16:27:40,397 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_energy Version: 5.0.1
2025-05-19 16:27:40,398 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e49ac801-ef8f-4634-8411-5fe51532d24a directory successfully created
2025-05-19 16:27:40,399 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_npt.edr to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e49ac801-ef8f-4634-8411-5fe51532d24a
2025-05-19 16:27:40,399 [MainThread  ] [INFO ]  gmx energy -f /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e49ac801-ef8f-4634-8411-5fe51532d24a/4G6K_npt.edr -o /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_e49ac801-ef8f-4634-8411-5fe51532d24a/4G6K_npt_PD.xvg -xvg none < c8bff021-b09b-46e0-bb62-40329280f890/instructions.in

2025-05-19 16:27:40,41

0

In [29]:
import plotly
from plotly import subplots
import plotly.graph_objs as go

# Read pressure and density data from file 
with open(output_npt_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

<a id="free"></a>
***
### Free Molecular Dynamics Simulation
Upon completion of the **two equilibration phases (NVT and NPT)**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released. The last step of the **protein** MD setup is a short, **free MD simulation**, to ensure the robustness of the system. 
- [Step 1](#mdStep1): Creating portable binary run file to run a **free MD simulation**.
- [Step 2](#mdStep2): Run short MD simulation of the **protein system**.
- [Step 3](#mdStep3): Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. 
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXRms](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rms) from **biobb_analysis.gromacs.gmx_rms** 
 - [GMXRgyr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rgyr) from **biobb_analysis.gromacs.gmx_rgyr** 
***

<a id="mdStep1"></a>
#### Step 1: Creating portable binary run file to run a free MD simulation

The **free** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **free MD simulation** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  integrator               = md
-  dt                       = 0.002 (ps)
-  nsteps                   = 50000

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, and a total of **50,000 md steps** (100ps).

*Please note that for the sake of time this tutorial is only running 100ps of free MD, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/08_MD.html) the simulated time was 1ns (1000ps).*

In [5]:
# https://github.com/alevil-gmx/workflow_template/blob/main/workflow_template/data/input/md_charmm36m.mdp
# ns_type -> ns-type
input_mdp_md = f"{MD_inp}md_charmm36m.mdp"

file=""";define                   =  -DPOSRES
;include                  = -I/home/villa/work/UseCaseI/forcefield
integrator               = md
tinit                    = 0
dt                       = 0.002
nsteps                   = 50000000   ; 100  ns
;nstcomm                  = 1
comm_grps                = system
nstxout                  = 0
nstvout                  = 0
nstfout                  = 0
nstlog                   = 100000
nstenergy                = 100000
nstxout-compressed       = 100000
compressed-x-grps        = system 
nstlist                  = 10
ns-type                  = grid
pbc                      = xyz
; neighbour
; rlist                   = 1.2 ; not used when cutoff-scheme = verlet
cutoff-scheme           = Verlet
; coulomb
coulombtype              = PME
rcoulomb                 = 1.2
fourierspacing           = 0.15 ;  For constant accuracy one should keep fourier-spacing/rcoulomb constant = 0.125
; vdw
vdwtype                  = Cut-off
vdw_modifier             = Force-switch
rvdw_switch              = 1.0  ; 
rvdw                     = 1.2
DispCorr                 = EnerPres ; while for lipid bilayer,  it's a difficult topic in the CHARMM force field. If one don't have lipids bi- or monolayers one should use it. 
;
constraints              = h-bonds
constraint_algorithm     = LINCS
; barostat
Pcoupl                   = Parrinello-Rahman   
tau_p                    = 5.0  
ref_p                    = 1.0 
compressibility          = 4.5e-5 
; thermostat
Tcoupl                   = v-rescale 
tc-grps                  = system
tau_t                    = 0.5 ; or 0.1 
ref_t                    = 298 """
with open(input_mdp_md, 'w') as f:
    f.write(file)

In [6]:
output_npt_gro = f'{MD_dir}{pdbCode}_npt.gro'
output_genion_top_zip = f'{MD_dir}{pdbCode}_genion_top.zip'
output_npt_cpt = f'{MD_dir}{pdbCode}_npt.cpt'

In [7]:
# Grompp: Creating portable binary run file for mdrun
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs

output_gppmd_tpr = f"{MD_dir}{pdbCode}_gppmd.tpr"

prop = {
    'mdp': {
        'nsteps' : 5_000_000, # 1 ns
        'nstxout' :  100_000
    },
    'gmx_lib' : f"{cwd}/{MD_inp}",
    'maxwarn' : 1
}

# Create and launch bb
grompp(input_gro_path=output_npt_gro, 
       input_top_zip_path=output_genion_top_zip, 
       input_mdp_path=input_mdp_md,
       output_tpr_path=output_gppmd_tpr, 
       input_cpt_path=output_npt_cpt, 
       properties=def_dict(prop))

2025-06-02 14:29:27,417 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.grompp Version: 5.0.0
2025-06-02 14:29:27,417 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_7c774527-51ed-42b4-8d4b-c98984906c7b directory successfully created
2025-06-02 14:29:27,421 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_npt.gro to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_7c774527-51ed-42b4-8d4b-c98984906c7b
2025-06-02 14:29:27,424 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_npt.cpt to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_7c774527-51ed-42b4-8d4b-c98984906c7b
2025-06-02 14:29:27,425 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/input/md_charmm36m.mdp to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_7c774527-51ed-42b4-8d4b-c98984906c7b
2025-06-02 14:29:27,429 [MainThread  ] [INFO ]  Extracting: /home/rchave

0

<a id="mdStep2"></a>
#### Step 2: Running short free MD simulation

In [9]:
output_gppmd_tpr

'data/antibody/1_MD/4G6K_gppmd.tpr'

In [ ]:
# Mdrun: Running free dynamics
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_md_trr = f'{MD_dir}{pdbCode}_md.trr'
output_md_gro = f'{MD_dir}{pdbCode}_md.gro'
output_md_edr = f'{MD_dir}{pdbCode}_md.edr'
output_md_log = f'{MD_dir}{pdbCode}_md.log'
output_md_cpt = f'{MD_dir}{pdbCode}_md.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppmd_tpr, 
      output_trr_path=output_md_trr, 
      output_gro_path=output_md_gro, 
      output_edr_path=output_md_edr, 
      output_log_path=output_md_log, 
      output_cpt_path=output_md_cpt,
      properties=def_dict())

2025-05-23 10:55:34,960 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.mdrun Version: 5.0.0
2025-05-23 10:55:34,961 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_32e09680-00c1-4234-9f57-42d5bb327143 directory successfully created
2025-05-23 10:55:34,963 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4G6K_gppmd.tpr to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_32e09680-00c1-4234-9f57-42d5bb327143
2025-05-23 10:55:34,963 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -o /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_32e09680-00c1-4234-9f57-42d5bb327143/4G6K_md.trr -s /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_32e09680-00c1-4234-9f57-42d5bb327143/4G6K_gppmd.tpr -c /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_32e09680-00c1-4234-9f57-42d5bb327143/4G6K_md.gro -e /home/rch

<a id="post"></a>
***
### Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **protein system** MD setup **resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep2): Generating a *dry* structure, **removing water molecules and ions** from the final snapshot of the MD setup pipeline.
- [Step 3](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**Building Blocks** used:
 - [GMXImage](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_image) from **biobb_analysis.gromacs.gmx_image** 
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str** 
***

<a id="ppStep1"></a>
#### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [15]:
output_md_trr = f'{MD_dir}{pdbCode}_md.trr'
output_md_gro = f'{MD_dir}{pdbCode}_md.gro'
output_md_edr = f'{MD_dir}{pdbCode}_md.edr'
output_md_log = f'{MD_dir}{pdbCode}_md.log'
output_md_cpt = f'{MD_dir}{pdbCode}_md.cpt'
output_gppmin_tpr = f'{MD_dir}{pdbCode}_gppmin.tpr'
output_gppmd_tpr  = f'{MD_dir}{pdbCode}_gppmd.tpr'
output_imaged_traj = f'{MD_dir}{pdbCode}_imaged_traj.trr'

In [10]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
from biobb_analysis.gromacs.gmx_image import gmx_image

# Create prop dict and inputs/outputs
output_imaged_traj = f'{MD_dir}{pdbCode}_imaged_traj.trr'
prop = {
    'center_selection':  'Protein',
    'output_selection': 'Protein',
    'pbc' : 'nojump',
}

# Create and launch bb
gmx_image(input_traj_path=output_md_trr,
         input_top_path=output_gppmin_tpr,
         output_traj_path=output_imaged_traj, 
          properties=def_dict(prop))

2025-06-04 09:56:23,481 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_image Version: 5.0.1
2025-06-04 09:56:23,482 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_233d0e41-c29c-4ff8-8fa9-ba511e85a105 directory successfully created
2025-06-04 09:56:23,504 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_md.trr to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_233d0e41-c29c-4ff8-8fa9-ba511e85a105
2025-06-04 09:56:23,508 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_gppmin.tpr to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_233d0e41-c29c-4ff8-8fa9-ba511e85a105
2025-06-04 09:56:23,509 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/5b2c

0

In [16]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
from biobb_analysis.gromacs.gmx_image import gmx_image

# Create prop dict and inputs/outputs
output_imaged_traj_rot = f'{MD_dir}{pdbCode}_imaged_traj_rot.trr'
prop = {
    'center_selection':  'Protein',
    'output_selection': 'Protein',
    'fit' : 'rot+trans',
    'center' : True
}

# Create and launch bb
gmx_image(input_traj_path=output_imaged_traj,
          input_top_path=output_gppmd_tpr,
          output_traj_path=output_imaged_traj_rot, 
          properties=def_dict(prop))

2025-06-04 09:58:15,625 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_image Version: 5.0.1
2025-06-04 09:58:15,626 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_3a28a3f7-d07b-45f8-8eb7-94c58c42f600 directory successfully created
2025-06-04 09:58:15,629 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_imaged_traj.trr to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_3a28a3f7-d07b-45f8-8eb7-94c58c42f600
2025-06-04 09:58:15,632 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_gppmd.tpr to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_3a28a3f7-d07b-45f8-8eb7-94c58c42f600
2025-06-04 09:58:15,633 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebo

0

<a id="ppStep2"></a>
#### Step 2: Generating the output *dry* structure.
**Removing water molecules and ions** from the resulting structure

In [17]:
# GMXTrjConvStr: Converting and/or manipulating a structure
#                Removing water molecules and ions from the resulting structure
#                The "dry" structure will be used as a topology to visualize 
#                the "imaged dry" trajectory generated in the previous step.
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str

# Create prop dict and inputs/outputs
output_dry_gro = f'{MD_dir}{pdbCode}_md_dry.gro'
prop = {
    'selection':  'Protein'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_md_gro,
         input_top_path=output_gppmd_tpr,
         output_str_path=output_dry_gro, 
          properties=def_dict(prop))

2025-06-04 09:58:19,257 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_trjconv_str Version: 5.0.1
2025-06-04 09:58:19,258 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_e852c090-819d-4925-b0cc-76f1cea32417 directory successfully created
2025-06-04 09:58:19,265 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_md.gro to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_e852c090-819d-4925-b0cc-76f1cea32417
2025-06-04 09:58:19,268 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_gppmd.tpr to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_e852c090-819d-4925-b0cc-76f1cea32417
2025-06-04 09:58:19,270 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks

0

<a id="ppStep3"></a>
#### Step 3: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of the [Post-processing step 2](#ppStep2)) as a topology.

In [20]:
# Show trajectory
view = nv.show_simpletraj(nv.SimpletrajTrajectory(output_imaged_traj_rot, output_dry_gro), gui=True)
view.add_licorice()
view

NGLWidget(max_frame=50)

<a id="mdStep3"></a>
#### Step 4: Checking free MD simulation results
Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [21]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against experimental structure (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_exp = f'{MD_dir}{pdbCode}_rms_exp.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_imaged_traj_rot,
         output_xvg_path=output_rms_exp, 
          properties=def_dict(prop))

2025-06-04 09:59:29,336 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_rms Version: 5.0.1
2025-06-04 09:59:29,337 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_3147a180-3886-404d-b921-9050a6f23b38 directory successfully created
2025-06-04 09:59:29,340 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_gppmin.tpr to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_3147a180-3886-404d-b921-9050a6f23b38
2025-06-04 09:59:29,343 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_imaged_traj_rot.trr to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_3147a180-3886-404d-b921-9050a6f23b38
2025-06-04 09:59:29,344 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/not

0

In [24]:
import plotly
import plotly.graph_objs as go

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )

trace = go.Scatter(
    x = x2,
    y = y2,
    name = 'RMSd vs exp'
)

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": trace,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (nm)")
                       )
}

plotly.offline.iplot(fig)


In [48]:
# GMXRgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

from biobb_analysis.gromacs.gmx_rgyr import gmx_rgyr

# Create prop dict and inputs/outputs
output_rgyr = f'{MD_dir}{pdbCode}_rgyr.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rgyr(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_imaged_traj_rot,
         output_xvg_path=output_rgyr, 
          properties=def_dict(prop))

2025-05-19 17:20:09,668 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_rgyr Version: 5.0.1
2025-05-19 17:20:09,669 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0e07ec5b-d1e8-4525-ab73-5d976a7f78a5 directory successfully created
2025-05-19 17:20:09,672 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_gppmin.tpr to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0e07ec5b-d1e8-4525-ab73-5d976a7f78a5
2025-05-19 17:20:09,675 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_imaged_traj_rot.trr to /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebooks/sandbox_0e07ec5b-d1e8-4525-ab73-5d976a7f78a5
2025-05-19 17:20:09,675 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_haddock/biobb_wf_haddock/notebook

0

In [49]:
import plotly
import plotly.graph_objs as go

# Read Rgyr data from file 
with open(output_rgyr,'r') as rgyr_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rgyr_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Radius of Gyration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Rgyr (nm)")
                       )
}

plotly.offline.iplot(fig)

#### Step 5. Clustering CDR loops

In [25]:
fixed_pdb = f'{MD_dir}{pdbCode}_fixed.pdb'

In [26]:
from anarcii import Anarcii

model = Anarcii(seq_type="antibody", mode="accuracy", verbose=True)
anarcii_stem = f'{MD_dir}4g6k_fixed-anarcii-imgt'
# results renunbers the file and returns the numbering
results = model.number(fixed_pdb)

Using device CUDA with 16 CPUs
Batch size: 32
	Speed is a balance of batch size and length diversity. Adjust accordingly. For a full explanation see:
 	wiki/FAQs#recommended-batch-sizes
 	Seqs all similar length (+/-5), increase batch size. Mixed lengths (+/-30), reduce.


Consider larger batch size for optimal GPU performance.

Length of sequence list: 2
Processing sequences in 1 chunks of 102400 sequences.
Processing chunk 1 of 1.

 2 Long sequences detected - running in sliding window. This is slow.
Max probability windows selected.

Making predictions on 1 batches.
PDBx model index, chain ID: 0, H
ANARCII chain type (score): H (30.58661460876465)
 Sequence length: 128
 Sequence: [((1, ' '), 'Q'), ((2, ' '), 'V'), ((3, ' '), 'Q'), ((4, ' '), 'L'), ((5, ' '), 'Q'), ((6, ' '), 'E'), ((7, ' '), 'S'), ((8, ' '), 'G'), ((9, ' '), 'P'), ((10, ' '), '-'), ((11, ' '), 'G'), ((12, ' '), 'L'), ((13, ' '), 'V'), ((14, ' '), 'K'), ((15, ' '), 'P'), ((16, ' '), 'S'), ((17, ' '), 'Q'), ((18, ' ')

In [27]:
from anarcii.output_data_processing.schemes_constants import schemes

# Show protein
view = nv.show_structure_file(anarcii_stem+'.pdb')
# https://www.imgt.org/IMGTScientificChart/Nomenclature/IMGT-FRCDRdefinition.html
view.add_ball_and_stick(selection='27-38', color='green')   #CDR1
view.add_ball_and_stick(selection='56-65', color='green')   #CDR2
view.add_ball_and_stick(selection='105-116', color='green') #CDR3
view

NGLWidget()

In [28]:
from biobb_gromacs.gromacs.make_ndx import make_ndx

# Create prop dict and inputs/outputs
loop_ndx = f'{MD_dir}{pdbCode}_loop.ndx'
sele = 'r 27-38 | r 56-65 | r 105-116\nname 10 Loop'

prop = { 
    'selection': sele
}

# Create and launch bb
make_ndx(input_structure_path=anarcii_stem+'.pdb',
         output_ndx_path=loop_ndx,
         properties=def_dict(prop))

2025-06-04 10:01:08,218 [MainThread  ] [INFO ]  Module: biobb_gromacs.gromacs.make_ndx Version: 5.0.0
2025-06-04 10:01:08,218 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_44e89ae1-9b2d-4859-8f0a-453810c765a9 directory successfully created
2025-06-04 10:01:08,219 [MainThread  ] [INFO ]  Copy: data/antibody/1_MD/4g6k_fixed-anarcii-imgt.pdb to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_44e89ae1-9b2d-4859-8f0a-453810c765a9
2025-06-04 10:01:08,220 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/8689a9b6-7c98-457b-b226-1e4101118029.stdin to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_44e89ae1-9b2d-4859-8f0a-453810c765a9
2025-06-04 10:01:08,220 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright make_ndx -f /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_44e89ae1-9b2d-4

0

In [29]:
# REMOVE
output_gppmin_tpr = f'{MD_dir}{pdbCode}_gppmin.tpr'
output_imaged_traj_rot = f'{MD_dir}{pdbCode}_imaged_traj_rot.trr'

In [30]:
from biobb_analysis.gromacs.gmx_cluster import gmx_cluster

output_pdb_cluster = f'{MD_dir}{pdbCode}_clusters.pdb'

prop = {
    'fit_selection': 'Loop',
    'output_selection': 'System',
#    'method': 'gromos',
    'method' : 'jarvis-patrick',
    'cutoff' : 0.1,
    'nofit' : True
}
gmx_cluster(input_structure_path=output_gppmin_tpr,
            input_traj_path=output_imaged_traj_rot,
            input_index_path=loop_ndx,
            output_pdb_path=output_pdb_cluster,
            properties=def_dict(prop))

2025-06-04 10:01:15,100 [MainThread  ] [INFO ]  Module: biobb_analysis.gromacs.gmx_cluster Version: 5.0.1
2025-06-04 10:01:15,101 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_0a9c657f-a61f-4555-8df9-51094e6d9ad3 directory successfully created
2025-06-04 10:01:15,104 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_gppmin.tpr to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_0a9c657f-a61f-4555-8df9-51094e6d9ad3
2025-06-04 10:01:15,106 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/data/antibody/1_MD/4G6K_imaged_traj_rot.trr to /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock/notebooks/sandbox_0a9c657f-a61f-4555-8df9-51094e6d9ad3
2025-06-04 10:01:15,107 [MainThread  ] [INFO ]  Copy: /home/rchaves/repo/ab_design/biobb_wf_antibody/biobb_wf_haddock

0

In [31]:
output_pdb_cluster = f'{MD_dir}{pdbCode}_clusters.pdb'

In [33]:
!code {output_pdb_cluster}

In [32]:
view = nv.show_structure_file(output_pdb_cluster, default_representation=False)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='all', color='modelindex')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

## Docking the generated structures with HADDOCK

In [ ]:
from biobb_haddock.haddock.haddock3_run import haddock3_run

In [34]:
# Obtain epitope
!arctic3d P01584 --db /home/rchaves/repo/arctic3d/db/swissprot --pdb_to_use 4I1B

/bin/bash: line 1: arctic3d: command not found


### Running HADDOCK3

In [ ]:
from biobb_haddock.haddock.common import read_cfg
from haddock.gear.config import load, save

In [39]:
final_cfg = load(haddock_config_path)
final_cfg['final_cfg']

{'run_dir': PosixPath('run1'),
 'mode': 'local',
 'ncores': 50,
 'postprocess': True,
 'clean': True,
 'molecules': [PosixPath('pdbs/4G6K_clean.pdb'),
  PosixPath('pdbs/4I1B_clean.pdb')],
 'topoaa.1': {},
 'rigidbody.1': {'ambig_fname': PosixPath('restraints/ambig-paratope-NMR-epitope.tbl'),
  'unambig_fname': PosixPath('restraints/antibody-unambig.tbl'),
  'sampling': 100},
 'caprieval.1': {'reference_fname': PosixPath('pdbs/4G6M_matched.pdb')},
 'seletop.1': {'select': 40},
 'flexref.1': {'tolerance': 5,
  'ambig_fname': PosixPath('restraints/ambig-paratope-NMR-epitope.tbl'),
  'unambig_fname': PosixPath('restraints/antibody-unambig.tbl')},
 'caprieval.2': {'reference_fname': PosixPath('pdbs/4G6M_matched.pdb')},
 'emref.1': {'tolerance': 5,
  'ambig_fname': PosixPath('restraints/ambig-paratope-NMR-epitope.tbl'),
  'unambig_fname': PosixPath('restraints/antibody-unambig.tbl')},
 'caprieval.3': {'reference_fname': PosixPath('pdbs/4G6M_matched.pdb')},
 'clustfcc.1': {'plot_matrix': True

In [21]:
save(load(wf_config)['final_cfg'], 'output_file_path.cfg')

In [36]:
from biobb_haddock.haddock.haddock3_run import haddock3_run

haddock_config_path        = f'{haddock_data}workflows/docking-antibody-antigen.cfg'
output_haddock_wf_data_zip = f'{out_path}wf.zip'  

haddock3_run(mol1_input_pdb_path  = ab_clean_pdb,
             mol2_input_pdb_path  = ag_clean_pdb,
             haddock_config_path        = haddock_config_path,
             output_haddock_wf_data_zip = output_haddock_wf_data_zip)

2025-05-05 10:38:18,057 [MainThread  ] [INFO ]  Module: biobb_haddock.haddock.haddock3_run Version: 5.0.0
2025-05-05 10:38:18,058 [MainThread  ] [INFO ]  /home/rchaves/repo/ab_design/biobb_wf_haddock/sandbox_8cb99d08-da81-4a4f-97e4-91f820d07e3d directory successfully created
2025-05-05 10:38:18,059 [MainThread  ] [INFO ]  Copy: ./data/antibody/HADDOCK3-antibody-antigen/pdbs/4G6K_clean.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/sandbox_8cb99d08-da81-4a4f-97e4-91f820d07e3d
2025-05-05 10:38:18,059 [MainThread  ] [INFO ]  Copy: ./data/antibody/HADDOCK3-antibody-antigen/pdbs/4I1B_clean.pdb to /home/rchaves/repo/ab_design/biobb_wf_haddock/sandbox_8cb99d08-da81-4a4f-97e4-91f820d07e3d
2025-05-05 10:38:18,060 [MainThread  ] [INFO ]  Copy: ./data/antibody/HADDOCK3-antibody-antigen/workflows/docking-antibody-antigen.cfg to /home/rchaves/repo/ab_design/biobb_wf_haddock/sandbox_8cb99d08-da81-4a4f-97e4-91f820d07e3d
/home/rchaves/repo/ab_design/biobb_wf_haddock/sandbox_8cb99d08-da81-4a4f-97

KeyError: 'haddock_step_name'

### Analysis of results

## PMX

In [ ]:
# https://mmb.irbbarcelona.org/biobb/workflows/tutorials/biobb_wf_pmx_tutorial
# https://github.com/bioexcel/biobb_wf_pmx_tutorial

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
